## Data exploration

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [ ]:
## Converting .csv file to a pandas df
df = pd.read_csv("../csv_files/development.csv")

In [ ]:
## Description of the data
df.describe()

In [ ]:
## Histogram of the 5 features on the first pad
pad_1 = df[["pmax[1]", "negpmax[1]", "area[1]", "tmax[1]", "rms[1]"]]
pad_1.hist(bins=100, figsize=(10,10))

In [ ]:
## Correlation matrix
import seaborn as sns
import matplotlib.pyplot as plt

correlation_matrix = df.corr()
plt.figure(figsize=(20,20))
sns.heatmap(correlation_matrix, cmap='coolwarm')
plt.show()

Looks like there are some pads/features that has low/none correlation with x and y values. Seems like it is pad (0, 7, 12, 15, 16, 17).  
Rms feature has also low correlation with x and y.

In [ ]:
## Removing pads with format: pads = ["0", "7", "12", ..]
def drop_pads(df, pads):
    cols_to_drop = [col for col in df.columns if any(idx in col for idx in pads)]
    df_removed = df.drop(cols_to_drop, axis=1)    
    return df_removed

df_copy = df 
df_copy.drop(["x", "y"], axis=1, inplace=True)
pads = ["0", "7", "12", "15", "16", "17"]

df_without_noise = drop_pads(df_copy,pads)

In [ ]:
## Visualization without pads (0, 7, 12, 15, 16, 17)
correlation_matrix = df_without_noise.corr()
plt.figure(figsize=(20,20))
sns.heatmap(correlation_matrix, cmap='coolwarm')
plt.show()

In [ ]:
def quantile2(dframe, lw=0.05, up=0.95, drop=True):
    tresholds = {}
    for col_name in dframe.columns:
        lw_tresh = dframe[col_name].quantile(lw)
        up_tresh = dframe[col_name].quantile(up)
        tresholds[col_name] = [lw_tresh, up_tresh]
    print(f"tresholds for {lw}, {up}: {tresholds}")
    initial_dim = dframe.shape
    for col_name in dframe.columns:
        if drop:
            dframe.drop(dframe[dframe[col_name] < tresholds[col_name][0]].index, inplace=True)
            dframe.drop(dframe[dframe[col_name] > tresholds[col_name][1]].index, inplace=True)
        else:
            dframe.loc[df[col_name] < tresholds[col_name][0], col_name] = tresholds[col_name][0]
            dframe.loc[df[col_name] > tresholds[col_name][1], col_name] = tresholds[col_name][1]

    new_dim = dframe.shape
    print(f"""
          initial dim:   {initial_dim}
          new dim:       {new_dim}
          a reduction of {((initial_dim[0]-new_dim[0])/initial_dim[0])*100}% of rows
          """)

In [ ]:
## Code for extracting most correlated features
def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(df, 10))

## Models

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

In [ ]:
# Loading the evaluation data
dev = pd.read_csv("../csv_files/development.csv")

In [ ]:
# Extracting the positions and removes the x and y column.
import numpy as np
pos_dev = dev[["x", "y"]]

## Dropping data from x and y 
dev = dev.drop(["x", "y"], axis=1)

In [ ]:
## Removing pads with format: pads = [0, 7, 12, ..]
def drop_pads(input_list, df):
    for i in input_list:
        columns_to_remove = df.filter(like=f'[{i}]').columns
        df = df.drop(columns=columns_to_remove)
    return df

remove_pads = [0, 7, 12, 15, 16, 17]
dev_interesting_data = drop_pads(remove_pads, dev)

In [ ]:
## Removing features rms
def drop_rms_features(df):
    # Extract columns that start with 'rms'
    rms_columns = [col for col in df.columns if not col.startswith('rms')]

    # Create a new DataFrame without 'rms' columns
    df_without_rms = df[rms_columns] 
    return df_without_rms

dev_interesting_data = drop_rms_features(dev_interesting_data)

In [ ]:
## Removing tmax feature
def drop_tmax_features(df):
    # Extract columns that start with 'rms'
    tmax_columns = [col for col in df.columns if not col.startswith('tmax')]

    # Create a new DataFrame without 'rms' columns
    df_without_tmax = df[tmax_columns] 
    return df_without_tmax

dev_interesting_data = drop_tmax_features(dev_interesting_data)

In [ ]:
## Z-transformation of the data. Remember to scale accordingly to training data for eval data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(dev_interesting_data)
 
dev_interesting_data = pd.DataFrame(scaler.transform(dev_interesting_data), columns=dev_interesting_data.columns)

In [ ]:
## Reducing the dataset to X percent of original size to speed up model testing
dev_interesting_data_sample = dev_interesting_data.sample(frac=0.20)
pos_dev_sample = pos_dev.loc[dev_interesting_data_sample.index]

In [ ]:
## Splitting into train and validation set
X_train, X_val, pos_train, pos_val = train_test_split(dev_interesting_data_sample, pos_dev_sample, test_size=0.2, random_state=42)

In [ ]:
## Code for analysing x and y data seperately
x_train_data = []
y_train_data = []
for i in range(len(pos_train)):
    x_train_data.append(pos_train.values[i][0])
    y_train_data.append(pos_train.values[i][1])

## Model testing

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
import math
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

## NOTE: Lines that are commented out are models and parameter that have been tried during the development.

# numb_trees = 70
# base_regressor = RandomForestRegressor(n_estimators=numb_trees, criterion="poisson", max_depth=30, max_features=0.35, bootstrap=True) 
# base_regressor = GradientBoostingRegressor() 
base_regressor = MLPRegressor(activation="logistic", alpha=0.0001, hidden_layer_sizes=(100, 100), learning_rate='constant', learning_rate_init=0.001, max_iter=1000, solver='adam', tol=0.001)
mult_regr = MultiOutputRegressor(base_regressor)

## Param grid for GridSearchCV for the MLPRegressor model
# param_grid = {
#     'hidden_layer_sizes': [(100,), (100,100), (100, 100, 100)],
#     'activation': ['relu'],
#     'solver': ['adam'],
#     'alpha': [0.1],
#     'learning_rate': ['constant', 'invscaling', 'adaptive'],
#     'max_iter': [500, 1000, 2000],
#     'learning_rate_init': [0.001],
#     'tol': [0.001, 0.0001]
# }

# grid_search = GridSearchCV(estimator=base_regressor, param_grid=param_grid, n_jobs=-1, cv=3, scoring="neg_mean_absolute_error")

# # Fit the GridSearchCV on the x data
# grid_search.fit(X_train, x_train_data)

# # Best parameters and best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# print("Best Parameters:", best_params)
# print("Best Score:", best_score)


mult_regr.fit(X_train, pos_train)
# mult_regr.fit(dev_interesting_data, pos_dev)
pos_pred = mult_regr.predict(X_val)

In [ ]:
# ## Plotting the feature importance for x and y-coordinate with the RandomForestRegressor using mean decrease in impurity. 
# importances_x = mult_regr_x.feature_importances_
# std = np.std([tree.feature_importances_ for tree in mult_regr_x.estimators_], axis=0)

# forest_importances_x = pd.Series(importances_x, index=dev_interesting_data_sample.columns)
# fig, ax = plt.subplots()
# forest_importances_x.plot.bar(yerr=std, ax=ax)
# ax.set_title("Feature importances for x-coordinate using MDI")
# ax.set_ylabel("Mean decrease in impurity")
# fig.tight_layout()

# importances_y = mult_regr_y.feature_importances_
# std = np.std([tree.feature_importances_ for tree in mult_regr_y.estimators_], axis=0)

# forest_importances_y = pd.Series(importances_y, index=dev_interesting_data_sample.columns)
# fig, ax = plt.subplots()
# forest_importances_y.plot.bar(yerr=std, ax=ax)
# ax.set_title("Feature importances for y-coordinate using MDI")
# ax.set_ylabel("Mean decrease in impurity")
# fig.tight_layout()

In [ ]:
# ## Plotting the feature by importance in decreasing order.
# # Convert the importances into a DataFrame
# feature_importances_x = pd.DataFrame({"feature": X_train.columns, "importance": importances_x})

# # Sort the DataFrame by importance
# feature_importances_x = feature_importances_x.sort_values("importance", ascending=False)

# # Plotting
# plt.figure(figsize=(10,6))
# plt.title("Feature Importances for x-coordinate")
# plt.bar(feature_importances_x["feature"], feature_importances_x["importance"], color="b")
# plt.xlabel("Features")
# plt.ylabel("Importance")
# plt.xticks(rotation="vertical")
# plt.show()


# # Convert the importances into a DataFrame
# feature_importances_y = pd.DataFrame({"feature": X_train.columns, "importance": importances_y})

# # Sort the DataFrame by importance
# feature_importances_y = feature_importances_y.sort_values("importance", ascending=False)

# # Plotting
# plt.figure(figsize=(10,6))
# plt.title("Feature Importances for y-coordinate")
# plt.bar(feature_importances_y["feature"], feature_importances_y["importance"], color="b")
# plt.xlabel("Features")
# plt.ylabel("Importance")
# plt.xticks(rotation="vertical")
# plt.show()

In [ ]:
# Metrics to evaluate model 
import sklearn.metrics as sm
import math
import numpy as np

def avg_euc_dist(pos_val, pos_pred):
    sum_square = 0
    for i in range(len(pos_val)):
        sum_square += math.sqrt((pos_val[i][0]-pos_pred[i][0])**2 + (pos_val[i][1]-pos_pred[i][1])**2)
    return sum_square/len(pos_val)    

def metrics_on_model(pos_val, pos_pred):
    print("Mean absolute error =", round(sm.mean_absolute_error(pos_val, pos_pred), 2)) 
    print("Mean squared error =", round(sm.mean_squared_error(pos_val, pos_pred), 2)) 
    print("Median absolute error =", round(sm.median_absolute_error(pos_val, pos_pred), 2)) 
    print("Explain variance score =", round(sm.explained_variance_score(pos_val, pos_pred), 2)) 
    print("R2 score =", round(sm.r2_score(pos_val, pos_pred), 2))
    print("Mean eucledian distance =", round(avg_euc_dist(pos_val, pos_pred), 2))

metrics_on_model(pos_val.to_numpy(), pos_pred)

## Using the model on the evaluation set

In [ ]:
# ev_data = pd.read_csv("../csv_files/evaluation.csv")

In [ ]:
# # Extracting the ID
# eval_id = ev_data["Id"]

# # Dropping the Id column from the ev_data
# ev_data = ev_data.drop(["Id"], axis=1)

In [ ]:
# # Formatting the position array to a string to be used in the .csv file 
# def pred_to_string(prediction_array):
#     pred_column = []
#     for i in range(len(prediction_array)):
#         pos_string = (str(prediction_array[i][0]) + "|" + str(prediction_array[i][1]))
#         pred_column.append(pos_string)
#     return pred_column
        

In [ ]:
# ## Preprocessing 
# remove_pads = [0, 7, 12, 15, 16, 17]
# ev_data = drop_pads(remove_pads, ev_data) # Remove pads
# ev_data = drop_rms_features(ev_data) # Remove rms feature 
# ev_data = drop_tmax_features(ev_data) # Remove t_max feature
# ev_data = pd.DataFrame(scaler.transform(ev_data), columns=ev_data.columns) # Z-transform with mean and std from training data

In [ ]:
# # Predicting the evaluation results
# mult_regr_eval = mult_regr.predict(ev_data)
# pos_pred = pred_to_string(mult_regr_eval) # Formatting the predictions 

In [ ]:
# # Creating a df and .csv file to be submitted. Saved in submission_file folder
# mult_reg_submission = pd.DataFrame({'Id': eval_id, 'Predicted': pos_pred})
# mult_reg_submission.to_csv("../DataScienceLab_Project/submission_files/MLP_all_data.csv", index=False)